In [1]:
#Below is my description of how to use my tools
import sqlite3, sys
import pandas as pd
import numpy as np

In [2]:
CPT_Terms = {
    "Oscopy": ["45378", "45380", "45381", "45382", "45383", "45384", "45385",
              "45388"],
    "Resection": ["44110", "44146", "44150", "44151", "44152", "44153", "44154", "44155", "44156",
                 "44157", "44158", "44159", "44160", "44204", "44205", "44206", "44207", "44208", "44209",
                 "44210", "44211", "44212"]
    }

poly_recog = {
    "Benign": ["211.3", "211.4"],
    "Malignant": ['152.0', '152.1', '152.2', '152.3', '152.4', 
                                       '152.5', '152.6', '152.7', '152.8', '152.9']
}


ICD_Codes = {
    "Oscopy": ["45.23"],
    "Resection": ["45.80", "45.81", "45.82", "45.83", "45.84", "45.85",
                  "45.86", "45.87", "45.88", "45.89", "45.70", "45.71",
                  "45.72", "45.73", "45.74", "45.75", "45.76", "45.77",
                  "45.78", "45.79"]
}

In [6]:
query = "SELECT * FROM sqlite_master WHERE type = 'table'"

con = sqlite3.connect("claims.db")

tabletest = pd.read_sql_query(query, con = con)



In [7]:
tabletest

,type,name,tbl_name,rootpage,sql
0,table,medical_headers,medical_headers,2,"CREATE TABLE ""medical_headers"" (\n""index"" INTE..."
1,table,medical_service_lines,medical_service_lines,103042,"CREATE TABLE ""medical_service_lines"" (\n""index..."


In [8]:
# Does a few things:
# Joins the database (slow) but once joined, it search across the data base for
# only those CPT codes that we care about and returns a dataframe of those CPT codes
# with the diagnositic codes, patient id, doctor id, and encounter keys

oscopy = CPT_Terms['Oscopy']
surgery = CPT_Terms['Resection']

placeholder_oscopy = "?" #To avoid sql injection
placeholders_oscopy = ", ".join([placeholder_oscopy] * len(oscopy))
placeholder_surgery = "?" #To avoid sql injection
placeholders_surgery = ", ".join([placeholder_surgery] * len(surgery))

query = "SELECT * from medical_service_lines INNER JOIN medical_headers ON medical_headers.encounter_key\
= medical_service_lines.encounter_key WHERE procedure IN (%s) OR procedure IN (%s)" %(placeholders_oscopy,
                                                                                             placeholders_surgery)

con = sqlite3.connect("claims.db")
msl = pd.read_sql_query(query, con = con, params = (*oscopy, *surgery))

con.close()

In [9]:
# First step is to find only those rows associated with a benign tumor
#Searches across the new dataframe based on the diagnostic codes associated with benign
#First pass is to grab the benign and the second is to remove those rows also associated with malignant

MslBenignFirst = msl[msl.isin(poly_recog["Benign"]).any(axis = 1)]
MslBenignOnly = MslBenignFirst[~MslBenignFirst.isin(poly_recog["Malignant"]).any(axis = 1)]

#This makes sure that there are no duplicate columns
MslBenignOnly = MslBenignOnly.loc[:,~MslBenignOnly.columns.duplicated()]

In [10]:
#Cleans the dataframe so that there are no duplicates on encounter_key and procedures
# This makes sure that each row is associated with one encounter and one procedure
# There is overlap between encounter and procedure - this is situations where they potentially
# had a colonoscopy and a surgery (it could also mean that they were coded several times for the same
# procedure however)

MslBenignOnly_Clean =  MslBenignOnly.drop_duplicates(subset = ["encounter_key", "procedure"])

In [11]:
MslBenignOnly_Clean

,index,encounter_key,procedure,modifier1,modifier2,units,line_charge,revenue_code,claim_type_code,icd_type,...,P11,P12,P13,P14,P15,patient_id,total_claim_charge_amount,bill_type_code,doctor_id,hospital_id
100,1780,ab78ff874e0a3fee2b4dea4a3a034f8d88762cf3ba321b...,45380,59,None,None,0000000000,0750,I,9,...,,,,,,9d25d542b0c6d0adf3085e885c96dcbec8eaf6d0445fd2...,000000475832,131,49eddb355a781d5bd9e89b349d5986485e861c78488783...,aa46fdeb05720bd6f3f386fde28299608ce5d2471ab0c0...
102,1789,c445984d6b1d16560ae1ac6f6a9af722f380f8c0a58a08...,45385,None,None,None,0000083900,0761,I,9,...,,,,,,a9e9da436a566303453bb5d1e07fcf29264266da670890...,000000306049,131,c06ad5fa65bbe453006e9d5188d6c1304185e318c0e2ec...,1c0a6598efb9ee9c9e2a875b5a8d327f161b6df34c20b9...
103,1796,b8103adee59d731cdf401e1c3e7b5ab353e08a09a56ed5...,45385,None,None,None,0000056500,0750,I,9,...,,,,,,94d0f5a1e7fad3c5a9b63997491a1bfc062d976dcd29c2...,000000266725,131,20b008ddbb1163412eb7aa3f521ca5f9f51d0eb04661d1...,0fe0d6c2742cddebfce7bd0751dbe21daade7fd3d1703e...
104,1801,b18a80df893de1699a324e28f0ad3cbff5f927aaf438dc...,45382,59,None,None,0000000100,0750,I,9,...,,,,,,2ae0c3a850e994f16804beb7513b770462b5af4a78ed5e...,000000099250,131,176d3aa8da81657e2e6a7c7b41511e78f7d13840fd837d...,39fc706761a86765ccfec3fecd02f4d6cd334f33ea0e8f...
152,2747,168ecb2e7f2c46aae393c10390e60f23a024029387476b...,45385,None,None,None,0000109980,0490,I,9,...,,,,,,b100724a60873d9c7a53f6038eb3ffc50d927e12fd85d3...,000000219960,831,83db17521d3df8c808c1b35cb93e178a72f117bed76273...,d42e76220d81c6d03334736f9087f565a52797d8cbf1df...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331470,6366510,7bce1a2256b26415cb5c181ca895ee12b1354f31df87c1...,45385,None,None,None,0000044444,0490,I,9,...,,,,,,d690600d7bc15b1e4d515ca92a547c478206b031e51e61...,000000044444,831,4731d1901db4a004f22e2a90c28f7ca092dfef6740edd4...,4dbe43969b784a0e38173a330326461f5e64bfff485730...
331551,6367965,d6ade2d8e255f8c677a80701d4899246ccf5839cb990f5...,45380,None,None,None,0000970200,0360,I,9,...,,,,,,80a98554cedd296512ac193dd4d451f6384a55fad73de4...,000001089100,131,6a2e3aaebe55e9def8332b072029527def806143e17fed...,9e6471382c8bcf5afdd45f56747b464e6226601a3468bc...
331562,6368222,a0fd1205dd7534993f147731c6320034d0bcead929de50...,45378,PT,None,None,0000229000,0490,I,9,...,,,,,,46761cad349c489bd6ccd38bcf4183a88ddf21bdb8dc18...,000000229000,831,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,19dc55ccb9dac2db4e2b26e2a97e175f90e3121a78ac4e...
331587,6368744,5e24da3658e1d68cc50887a129b5e0b1584f29c55db60a...,45384,PT,None,None,0000321500,0750,I,9,...,,,,,,2ff34054fe91ef424479c8b582e72fe49a229a4a77f7ad...,000000479400,131,439a3c62f78b55b5604ed2032871944bb69233ac82589d...,4808dbf9a77befdac22ea6791c996a7f95bd51183ecd82...


In [25]:
###################################################
################ RATE CONSTRUCTION ################
###################################################

In [23]:
# Now I go about constructing the rule: Dr involved with Colonoscopy should not
# refer for surgery if they the patient is benign - here I use encounter instead of patient
# because there can be multiple encounters per patient - you want to know what the doctor is doing
# per treatment rather than per person - an admitted weakness is this does not
# take into account the history of the patients - maybe they had malignant polyps in the past
# so it makes sense for them to have surgery on all of their polyps? 

In [12]:
#I now search across the cleaned dataframe for the CPT codes again
# to divide the dataset by the CPT codes: 1) for surgery and 2) for colonoscopy

Msl_SurgeryOnly = MslBenignOnly_Clean[MslBenignOnly_Clean.procedure.isin(CPT_Terms["Resection"])]
Msl_OscopyOnly = MslBenignOnly_Clean[MslBenignOnly_Clean.procedure.isin(CPT_Terms["Oscopy"])]

#Again need to drop any duplicates - it is here that I account for multiple CPT codes for same encounter
# and same procedure
Msl_OscopyOnly_cleaned = Msl_OscopyOnly.drop_duplicates(subset = ["encounter_key", "patient_id"])
Msl_SurgeryOnly_cleaned = Msl_SurgeryOnly.drop_duplicates(subset = ["encounter_key", "patient_id"])

In [13]:
#Next step is to grab all of the doctors who performed a colonoscopy
# and see how many of them also performed a surgery on a patient with a benign polyp

OscopyDr = list(Msl_OscopyOnly_cleaned.doctor_id)

In [15]:
#NUMERATOR CONSTRUCTION

# I then check to see how many of the doctors involved in a colonoscopy also did surgery

OscopyDrThatDidSurgery = Msl_SurgeryOnly_cleaned[Msl_SurgeryOnly_cleaned.doctor_id.isin(OscopyDr)]

# If you group this new dataframe by doctor_id and count - you can see how many
# doctors (612) were found to have done surgery - this is the numerator for the violation rate

NumeratorCount = OscopyDrThatDidSurgery.groupby("doctor_id", as_index = False)["encounter_key"].count()

# What you find is that there were 612 doctors involved with both surgery and a colonoscopy
# These were doctors who did surgery on a patient with a benign polyp and have
# been binned as "bad doctors"

BadDoctors = OscopyDrThatDidSurgery.drop_duplicates(subset = ["doctor_id"])
BadDoctors_ID = list(BadDoctors.doctor_id)



In [16]:
#DENOMINATOR CONSTRUCTION

# I then check to see how many times the doctors had the chance to not break the violate rate:

#Using BadDoctors_ID that I grab from above - I search through the colonoscopy records

ViolationRuleOpporunities = Msl_OscopyOnly_cleaned[Msl_OscopyOnly_cleaned.doctor_id.isin(BadDoctors_ID)]

#We find that in total (for all doctors) - they had the opportunity to not violate the rule for 5567 encounters

#If we groupby dr and count the number of encounters - we can construct the Denominator for our rate
DenominatorCount = ViolationRuleOpporunities.groupby("doctor_id", as_index = False )["encounter_key"].count()

                       

In [17]:
#VIOLATION RATE CONSTRUCTION

#With the above NumeratorCount and DenominatorCount - I can now construct the violation rate
RateConstructing = pd.merge(NumeratorCount, DenominatorCount, on = "doctor_id")
RateConstructing = RateConstructing.rename(columns = {"doctor_id" : "Doctor", "encounter_key_x" : "Numerator",\
                       "encounter_key_y" : "Denominator" })

RateConstructing["ViolationRate"] = RateConstructing["Numerator"]/RateConstructing["Denominator"]
RateConstructing

,Doctor,Numerator,Denominator,ViolationRate
0,0053f157450476af69c501f39e8780d7cea30aafbfb9a2...,1,4,0.250000
1,01324f85867eee3b706f934c90d2cf1b9aa998b8401e0b...,1,2,0.500000
2,0139ffe055a33b965049c60a0361ca6960ab1cd5eb0ec0...,5,15,0.333333
3,0234034ce12f711ec2a049728a93c51607adea0b974493...,2,13,0.153846
4,0337e0b39905ed3a61412dd9567ab6592f2039f1737eac...,3,7,0.428571
...,...,...,...,...
607,fdcbad76251190e6080f95f043482ee6adc1c3deb14577...,1,2,0.500000
608,fe72412295b13dce63790ab33a408686d04e3126cb85a2...,5,11,0.454545
609,fea12e63b3cd1f8b158b65301834a8eff0954f5fdda421...,3,6,0.500000
610,fea33e00602fb5470626ead40eef6af4f160e8c5a94465...,1,4,0.250000


In [18]:
#VIOLATION RATE FOR GOOD DOCTORS

#Need to do similar with the good doctors as well - those that did not do surgery on 
# patients with a benign polyp

#The number of doctors can be found by using the "Bad" Doctors ID and doing a 'not' search
# on the Colonoscopy records

OscopyDoctorNoSurgery = Msl_OscopyOnly_cleaned[~Msl_OscopyOnly_cleaned.doctor_id.isin(BadDoctors_ID)]

#If if you look at the "doctor_id" you can see the number of "Good" doctors - or those who do not break the rule
# This number is 613

GoodDr = list(OscopyDoctorNoSurgery.doctor_id.unique())

#Below also service as the violation rule opporutnties - the number of opporunities the good dr had to 
# violate the rule
VROppo_GoodDr = Msl_OscopyOnly_cleaned[~Msl_OscopyOnly_cleaned.doctor_id.isin(BadDoctors_ID)]\
.groupby("doctor_id", as_index = False)["encounter_key"].count()

#I am now inserting 0s into the Numerator since they did not violate the rule
VROppo_GoodDr.insert(1, 'Numerator', 0)

#Renaming the columns and then creating the ViolationRate
VROppo_GoodDr = VROppo_GoodDr.rename(columns = {"doctor_id" : "Doctor", 
                       "encounter_key" : "Denominator" })
VROppo_GoodDr["ViolationRate"] = VROppo_GoodDr["Numerator"]/VROppo_GoodDr["Denominator"]

VROppo_GoodDr

,Doctor,Numerator,Denominator,ViolationRate
0,0067c410b32482e2e46323b01a592300a87c9ae54b5ab4...,0,1,0.0
1,007a1afad99071d1f2cc5baeacaee818a604d689744e55...,0,9,0.0
2,01469d5cbaa6e581fd5cfb42d19dd9f2edfc52178924e8...,0,2,0.0
3,02b936724394d86f0950e1247bd61abeed9f65a708ff23...,0,4,0.0
4,032d9bc6b4ecc95af33616d413212769de3950d87d119b...,0,7,0.0
...,...,...,...,...
608,fdfd978f50b2f866104e47d7e0341dbd7e6aef24b9b983...,0,2,0.0
609,fe4815e390f0aaa0980d9bde7f7abb5504b2fb81649c0b...,0,4,0.0
610,fed826b68a18b292061647510e3af421f6363fa9775c0b...,0,3,0.0
611,ff40d814b56fdfb00155cb8056067d677e321f33910f1e...,0,1,0.0


In [19]:
# BINDING TOGETHER THE FULL DR VIOLATION RATE

#Now I just need to append these values together
ViolationRateFull = RateConstructing.append(VROppo_GoodDr)

#I now have the full violation rate
ViolationRateFull

,Doctor,Numerator,Denominator,ViolationRate
0,0053f157450476af69c501f39e8780d7cea30aafbfb9a2...,1,4,0.250000
1,01324f85867eee3b706f934c90d2cf1b9aa998b8401e0b...,1,2,0.500000
2,0139ffe055a33b965049c60a0361ca6960ab1cd5eb0ec0...,5,15,0.333333
3,0234034ce12f711ec2a049728a93c51607adea0b974493...,2,13,0.153846
4,0337e0b39905ed3a61412dd9567ab6592f2039f1737eac...,3,7,0.428571
...,...,...,...,...
608,fdfd978f50b2f866104e47d7e0341dbd7e6aef24b9b983...,0,2,0.000000
609,fe4815e390f0aaa0980d9bde7f7abb5504b2fb81649c0b...,0,4,0.000000
610,fed826b68a18b292061647510e3af421f6363fa9775c0b...,0,3,0.000000
611,ff40d814b56fdfb00155cb8056067d677e321f33910f1e...,0,1,0.000000
